In [ ]:
pip install seaborn

In [ ]:
pip install openpyxl

In [ ]:
pip install dython

In [ ]:
pip install imblearn

In [7]:
#import necessary libraries
import pandas as pd
import numpy as np
import openpyxl
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score , confusion_matrix , classification_report
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import RUSBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_val_score,cross_val_predict
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import svm, tree, metrics
from sklearn.feature_selection import *
from sklearn.model_selection import *

In [8]:
df = pd.read_excel('Parkinson1.xlsx')
df

,SUBJ_ID,VISIT_NAME,Source Of Information (PTCGBOTH),Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Cognitive State (COGSTATE),Level Of Confidence Cognitive Diagnosis (COGDXCL),Review Neuropsychological Tests (RVWNPSY),Age,Ethnicity,...,Supine Heart Rate (beats/min) (HRSUP),Standing Blood Pressure - Systolic (mmHg) (SYSSTND),Standing Blood Pressure - Diastolic (mmHg) (DIASTND),Standing Heart Rate (beats/min) (HRSTND),Serum IGF-1 (ng/mL),p-Tau181P (pg/ml),CSF Hemoglobin (ng/ml) (Textual),CSF Alpha-synuclein (pg/ml),Abeta 42 (pg/ml),Total tau (pg/ml)
0,3000,Month 60,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,70,Not Hispanic or Latino,...,60.0,130.0,70.0,60.0,170.1,16.1,NaN,2272.32,408.1,54.3
1,3001,Month 60,Patient,Yes,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,65,Not Hispanic or Latino,...,63.0,137.0,93.0,81.0,211.0,30.1,Below Detection Limit,1976.82,435.4,45.9
2,3002,Month 60,Patient,Yes,No,1 - Normal Cognition (PD-NC),50 Pct - 89 Pct,No,68,Not Hispanic or Latino,...,64.0,130.0,84.0,68.0,126.2,18.5,NaN,2972.06,204.1,99.3
3,3003,Month 60,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,57,Not Hispanic or Latino,...,60.0,110.0,70.0,68.0,142.8,8.2,NaN,1426.42,310.8,27.6
4,3004,Month 60,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,60,Not Hispanic or Latino,...,48.0,150.0,86.0,48.0,165.4,11.5,Below Detection Limit,1352.57,374.0,30.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,85242,Month 03,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,Yes,63,Not Hispanic or Latino,...,68.0,124.0,82.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN
893,90456,Month 24,Patient and Caregiver,Yes,No,2 - Mild Cognitive Impairment (PD-MCI),50 Pct - 89 Pct,No,82,Not Hispanic or Latino,...,62.0,168.0,102.0,68.0,NaN,NaN,NaN,NaN,NaN,NaN
894,91097,Month 24,Patient,No,No,1 - Normal Cognition (PD-NC),90 Pct - 100 Pct,No,67,Hispanic or Latino,...,64.0,120.0,80.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN
895,92490,Month 12,Patient,No,No,1 - Normal Cognition (PD-NC),50 Pct - 89 Pct,No,72,Not Hispanic or Latino,...,80.0,133.0,88.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print("============= Data Summary =============")
print("Missing Values: " ,np.sum(df.isnull().any(axis=1)))
print("Shape: ", df.shape)
print("Columns: ", len(df.columns))
print("Data types:")
print(df.dtypes)
     

============= TRAIN SET =============
Missing Values:  868
Shape:  (897, 382)
Columns:  382
Data types:
SUBJ_ID                                                int64
VISIT_NAME                                            object
Source Of Information (PTCGBOTH)                      object
Experienced Cognitive Decline (COGDECLN)              object
Functional Impairment Due To Cognitive (FNCDTCOG)     object
                                                      ...   
p-Tau181P (pg/ml)                                    float64
CSF Hemoglobin (ng/ml) (Textual)                      object
CSF Alpha-synuclein (pg/ml)                          float64
Abeta 42 (pg/ml)                                     float64
Total tau (pg/ml)                                    float64
Length: 382, dtype: object


In [10]:
#We need the types to be categorical, so we can correlate them 
#We convert them to categorical  
df[[col for col in df.columns if df[col].dtypes == object]] = df[[col for col in df.columns if df[col].dtypes == object]].astype('category')#.cat.codes

In [11]:
cat_cols = [col for col in df.columns if df[col].dtype == 'category']

In [12]:
for col in cat_cols:
    df[col] = df[col].cat.codes

In [8]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897 entries, 0 to 896
Columns: 382 entries, SUBJ_ID to Total tau (pg/ml)
dtypes: float64(116), int16(15), int64(2), int8(249)
memory usage: 1.0 MB


In [13]:
#Fill values with value of previous cell
df = df.ffill()

In [14]:
#Drop non assigned values
df = df.dropna()

In [ ]:
#Filter the data and keep the usefull attributes which have correlation greater than 0.2
dfCorr = corr_mat
filteredDf = dfCorr[(dfCorr >= .2) & (dfCorr !=1.000)]
filteredDf.to_csv('Filtered.csv')
filteredDf


In [ ]:
#Remove columns which have less than 0.2 correlatin with our target class 
to_drop = [column for column in corr_mat.columns if abs(corr_mat['Cognitive State (COGSTATE)'][column]) <= 0.2]

In [ ]:
#Drop the columns not included in the filtering process
df1 = df.drop(to_drop, axis=1)

In [15]:
#We seperate the "target" attribute from the rest 
X = df.drop('Cognitive State (COGSTATE)', axis=1)
#We use Cognitive State as "target" attribute 
y = df['Cognitive State (COGSTATE)']

In [16]:
y

1      0
2      0
3      0
4      0
5      0
      ..
892    0
893    1
894    0
895    0
896    0
Name: Cognitive State (COGSTATE), Length: 896, dtype: int8

In [17]:
X

,SUBJ_ID,VISIT_NAME,Source Of Information (PTCGBOTH),Experienced Cognitive Decline (COGDECLN),Functional Impairment Due To Cognitive (FNCDTCOG),Level Of Confidence Cognitive Diagnosis (COGDXCL),Review Neuropsychological Tests (RVWNPSY),Age,Ethnicity,Race,...,Supine Heart Rate (beats/min) (HRSUP),Standing Blood Pressure - Systolic (mmHg) (SYSSTND),Standing Blood Pressure - Diastolic (mmHg) (DIASTND),Standing Heart Rate (beats/min) (HRSTND),Serum IGF-1 (ng/mL),p-Tau181P (pg/ml),CSF Hemoglobin (ng/ml) (Textual),CSF Alpha-synuclein (pg/ml),Abeta 42 (pg/ml),Total tau (pg/ml)
1,3001,6,1,1,0,3,0,65,1,4,...,63.0,137.0,93.0,81.0,211.0,30.1,0,1976.82,435.4,45.9
2,3002,6,1,1,0,2,0,68,1,4,...,64.0,130.0,84.0,68.0,126.2,18.5,-1,2972.06,204.1,99.3
3,3003,6,1,0,0,3,0,57,1,4,...,60.0,110.0,70.0,68.0,142.8,8.2,-1,1426.42,310.8,27.6
4,3004,6,1,0,0,3,0,60,1,4,...,48.0,150.0,86.0,48.0,165.4,11.5,0,1352.57,374.0,30.8
5,3008,5,1,0,0,2,0,82,1,4,...,64.0,150.0,84.0,76.0,65.8,20.7,0,3720.71,224.0,90.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892,85242,0,1,0,0,3,1,63,1,4,...,68.0,124.0,82.0,78.0,103.0,15.9,-1,3319.59,349.3,54.2
893,90456,3,2,1,0,2,0,82,1,4,...,62.0,168.0,102.0,68.0,103.0,15.9,-1,3319.59,349.3,54.2
894,91097,3,1,0,0,3,0,67,0,3,...,64.0,120.0,80.0,64.0,103.0,15.9,-1,3319.59,349.3,54.2
895,92490,2,1,0,0,2,0,72,1,4,...,80.0,133.0,88.0,84.0,103.0,15.9,-1,3319.59,349.3,54.2


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [19]:
X_train.shape

(627, 381)

In [20]:
y_train.shape

(627,)

In [21]:
oversample = SMOTE(k_neighbors=2)
X_train , y_train = oversample.fit_resample(X_train,y_train)
y_train = pd.Series(y_train)
y_train.value_counts()

0    520
1    520
Name: Cognitive State (COGSTATE), dtype: int64

In [22]:
clf = RUSBoostClassifier(random_state=0)
clf.fit(X_train, y_train)

RUSBoostClassifier(random_state=0)

In [23]:
y_pred1 = clf.predict(X_test)


In [28]:
report = classification_report(y_test, y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       221
           1       0.71      0.75      0.73        48

    accuracy                           0.90       269
   macro avg       0.83      0.84      0.83       269
weighted avg       0.90      0.90      0.90       269



In [29]:
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)

In [30]:
model = abc.fit(X_train, y_train)


In [31]:
y_pred = model.predict(X_test)


In [32]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       221
           1       0.71      0.75      0.73        48

    accuracy                           0.90       269
   macro avg       0.83      0.84      0.83       269
weighted avg       0.90      0.90      0.90       269



In [34]:
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)
preds = xgb_cl.predict(X_test)
accuracy_score(y_test, preds)

/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[22:09:21] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.8921933085501859

In [35]:
print(classification_report(y_test,preds))


              precision    recall  f1-score   support

           0       0.94      0.93      0.93       221
           1       0.69      0.73      0.71        48

    accuracy                           0.89       269
   macro avg       0.81      0.83      0.82       269
weighted avg       0.90      0.89      0.89       269

